In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime
import ast
from itertools import chain

In [10]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'CMX'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 0

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [2]:
clients = get_fresh_query_result("https://internal-redash.federate.frubana.com/",164882,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [6]:
target = 300

In [7]:
clients = clients.loc[clients.total_spend < target]

In [8]:
clients

,customer_id,total_spend
2,-264814,299.920000
8,276732542,58.510000
13,112287806,260.671333
17,39715928,279.920000
29,310519467,228.410000
...,...,...
6784,131730014,23.910000
6785,235367162,16.800000
6786,238659537,19.010000
6787,115070275,204.480000


In [12]:
from sklearn.model_selection import train_test_split 

def create_groups(clients):
    
    customers = clients.customer_id.unique()
    
    customer_control, customer_test = train_test_split(customers, test_size = 0.50)
    
    alpha=0.01
    aux=0
    while abs(1-(clients[clients.customer_id.isin(customer_control)]['total_spend'].mean()/clients[clients.customer_id.isin(customer_test)]['total_spend'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [13]:
global_segment_list = [] #//for procesing
offer_name = []

control, test = create_groups(clients)

offer_name.append("TEST_FRUVER_AZUCAR_"+todays_date+"_"+city_code)
d = {'name':"TEST_FRUVER_AZUCAR_"+todays_date+"_"+city_code, 'customersIds': list(test)}
e = {'name':"TEST_FRUVER_AZUCAR_"+todays_date+"_"+city_code+"_CONTROL", 'customersIds': list(control)}

global_segment_list.append(d)
global_segment_list.append(e)

In [14]:
import requests
import json

url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [18]:
pd.DataFrame(global_segment_list).to_excel("test_cmx_prueba.xlsx",index=False)